### Play with networktables to see how the API is organized  - 20220313 CJH

In [295]:
import time
import networktables


In [296]:
ntinst = networktables.NetworkTablesInstance.getDefault()

#### can give nt multiple servers to try

In [297]:
servers = ["127.0.0.1", "10.24.29.2"] #, "roboRIO-2429-FRC.local"]  # need to add the USB one here
ntinst.startClient(servers=servers)

#### if it connects you can get the connection info - may have to wait a few seconds though

In [298]:
connections = ntinst.getConnections()

In [299]:
print(f'ID:{connections[0].remote_id}\tIP:{connections[0].remote_ip}')

ID:Robot	IP:127.0.0.1


#### looks like all entries are given their own subtable

In [300]:
ntinst.

SyntaxError: invalid syntax (1/ipykernel_24920/728264659.py, line 1)

In [301]:
table = ntinst.getGlobalTable()  # also NetworkTables.getTable('/')
table

In [302]:
subtables = table.getSubTables()
type(subtables)

list

In [303]:
len(subtables)

240

In [304]:
set(subtables)

{'FMSInfo', 'LiveWindow', 'Ramsete', 'Rev', 'SmartDashboard'}

In [305]:
s = table.getSubTable('SmartDashboard')

In [306]:
set(s.getSubTables())

{'',
 'Auto Rotate IMU',
 'AutoFetchBall',
 'AutoRotateSparkmax',
 'Field',
 'Tune Sparkmax',
 'auto lower group',
 'auto pickup',
 'auto shoot',
 'auto stage two',
 'autonomous routines',
 'autonomous_ramsete',
 'hold to feed',
 'intake motor',
 'intake piston',
 'path velocity',
 'ramsete path',
 'toggle compressor',
 'toggle shooter'}

In [307]:
for st in set(subtables):
    t = table.getSubTable(st)
    print(t.getName())
    

AttributeError: '_pyntcore._ntcore.NetworkTable' object has no attribute 'getName'

In [308]:
unique = list(set(subtables))
frequency = {}
for item in unique:
    frequency[item] = subtables.count(item)
print("Frequency of items: ", frequency)

Frequency of items:  {'SmartDashboard': 117, 'LiveWindow': 92, 'Rev': 14, 'FMSInfo': 9, 'Ramsete': 8}


In [309]:
r = table.getSubTable('Ramsete')

#### get all entries - not sure if this is the fastest way to do it

In [310]:
entries = ntinst.getEntries('/', types=0)

In [352]:
sorted_tree = sorted([e.getName() for e in entries])
sorted_tree

['/FMSInfo/.type',
 '/FMSInfo/EventName',
 '/FMSInfo/FMSControlData',
 '/FMSInfo/GameSpecificMessage',
 '/FMSInfo/IsRedAlliance',
 '/FMSInfo/MatchNumber',
 '/FMSInfo/MatchType',
 '/FMSInfo/ReplayNumber',
 '/FMSInfo/StationNumber',
 '/LiveWindow/.status/LW Enabled',
 '/LiveWindow/Ungrouped/.type',
 '/LiveWindow/Ungrouped/AnalogInput[0]/.name',
 '/LiveWindow/Ungrouped/AnalogInput[0]/.type',
 '/LiveWindow/Ungrouped/AnalogInput[0]/Value',
 '/LiveWindow/Ungrouped/Climber/.command',
 '/LiveWindow/Ungrouped/Climber/.default',
 '/LiveWindow/Ungrouped/Climber/.hasCommand',
 '/LiveWindow/Ungrouped/Climber/.hasDefault',
 '/LiveWindow/Ungrouped/Climber/.name',
 '/LiveWindow/Ungrouped/Climber/.type',
 '/LiveWindow/Ungrouped/Compressor[0]/.name',
 '/LiveWindow/Ungrouped/Compressor[0]/.type',
 '/LiveWindow/Ungrouped/Compressor[0]/Enabled',
 '/LiveWindow/Ungrouped/Compressor[0]/Pressure switch',
 '/LiveWindow/Ungrouped/DifferentialDrive[1]/.actuator',
 '/LiveWindow/Ungrouped/DifferentialDrive[1]/.name

#### why do i have to take what was a table and get a flat list?

In [312]:
nt_dict = {}
for subtable in sorted(set(subtables)):
    child = [entry[len(subtable)+2:] for entry in sorted_tree if subtable in entry]
    nt_dict.update({subtable:child})
#nt_dict

#### data dump from networktables

In [319]:
for item in sorted_tree:
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    # value = val.getDouble() if val.isDouble() else val.getBoolean() if val.isBoolean else val.getString if val.isString else ''
    value = entry_val.value()
    print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  Levels:{name[1:].split("/")}')

Name: /FMSInfo/.type                                               Value:FMSInfo              	 Age:     18  Type: NetworkTableType.kString  Levels:['FMSInfo', '.type']
Name: /FMSInfo/EventName                                           Value:                     	 Age:     18  Type: NetworkTableType.kString  Levels:['FMSInfo', 'EventName']
Name: /FMSInfo/FMSControlData                                      Value:32.0                 	 Age:     18  Type: NetworkTableType.kDouble  Levels:['FMSInfo', 'FMSControlData']
Name: /FMSInfo/GameSpecificMessage                                 Value:                     	 Age:     18  Type: NetworkTableType.kString  Levels:['FMSInfo', 'GameSpecificMessage']
Name: /FMSInfo/IsRedAlliance                                       Value:True                 	 Age:     18  Type: NetworkTableType.kBoolean  Levels:['FMSInfo', 'IsRedAlliance']
Name: /FMSInfo/MatchNumber                                         Value:0.0                  	 Age:     18  Type: Netw

could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address


In [353]:
t = ntinst.getEntry(sorted_tree[100])

In [356]:
t= ntinst.getEntry('/SmartDashboard/shooter_rpm')
t

In [322]:
[item for item in dir(t) if '_' not in item ]

['Flags',
 'addListener',
 'clearFlags',
 'clearPersistent',
 'delete',
 'exists',
 'forceSetBoolean',
 'forceSetBooleanArray',
 'forceSetDouble',
 'forceSetDoubleArray',
 'forceSetRaw',
 'forceSetString',
 'forceSetStringArray',
 'forceSetValue',
 'getBoolean',
 'getBooleanArray',
 'getDouble',
 'getDoubleArray',
 'getFlags',
 'getHandle',
 'getInfo',
 'getInstance',
 'getLastChange',
 'getName',
 'getRaw',
 'getString',
 'getStringArray',
 'getType',
 'getValue',
 'isPersistent',
 'removeListener',
 'setBoolean',
 'setBooleanArray',
 'setDefaultBoolean',
 'setDefaultBooleanArray',
 'setDefaultDouble',
 'setDefaultDoubleArray',
 'setDefaultRaw',
 'setDefaultString',
 'setDefaultStringArray',
 'setDefaultValue',
 'setDouble',
 'setDoubleArray',
 'setFlags',
 'setPersistent',
 'setRaw',
 'setString',
 'setStringArray',
 'setValue',
 'value']

In [323]:
t.getName()

'/LiveWindow/Ungrouped/navX-Sensor[4]/Value'

In [324]:
info = t.getInfo()

In [325]:
[item for item in dir(info) if '_' not in item ]

['entry', 'flags', 'name', 'type']

In [326]:
t.getType()

<NetworkTableType.kDouble: 2>

In [327]:
t.getLastChange()

1647237228607517

In [330]:
t.time()

AttributeError: '_pyntcore._ntcore.NetworkTableEntry' object has no attribute 'time'

In [328]:
val = t.getValue()

In [329]:
[item for item in dir(val) if '_' not in item ]

['getBoolean',
 'getBooleanArray',
 'getDouble',
 'getDoubleArray',
 'getFactoryByType',
 'getRaw',
 'getRpc',
 'getString',
 'getStringArray',
 'isBoolean',
 'isBooleanArray',
 'isDouble',
 'isDoubleArray',
 'isRaw',
 'isRpc',
 'isString',
 'isStringArray',
 'isValid',
 'makeBoolean',
 'makeBooleanArray',
 'makeDouble',
 'makeDoubleArray',
 'makeRaw',
 'makeRpc',
 'makeString',
 'makeStringArray',
 'makeValue',
 'time',
 'type',
 'value']

In [187]:
val.getDouble()

281.15814208984375

In [333]:
val.last_change()

1647237228607517

In [331]:
val.time()

1647237228607517

In [336]:
val.time??

Docstring:
time(self: _pyntcore._ntcore.Value) -> int

Get the creation time of the value.

:returns: The time, in the units returned by nt::Now().
Type:      method


could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve ro

In [202]:
val.value()

281.15814208984375

In [276]:
# generate the dictionary
d={}
levels = [s[1:].split('/') for s in sorted_tree]
for path in levels:
    current_level = d
    for part in path:
        if part not in current_level:
            current_level[part] = {}
        current_level = current_level[part]
#d

In [263]:
levels[100]

['LiveWindow', 'Ungrouped', 'navX-Sensor[4]', 'Value']

In [269]:
d['LiveWindow']['Ungrouped']['navX-Sensor[4]']['Value']

10

In [283]:
for item in sorted_tree:
    value = ntinst.getEntry(item).getValue().value()
    levels = item[1:].split('/')
    if len(levels) == 2:
        d[levels[0]][levels[1]] = value
    elif len(levels) == 3:
        d[levels[0]][levels[1]][levels[2]] = value
    elif len(levels) == 4:
        d[levels[0]][levels[1]][levels[2]][levels[3]] = value

In [284]:
d

{'FMSInfo': {'.type': 'FMSInfo',
  'EventName': '',
  'FMSControlData': 33.0,
  'GameSpecificMessage': '',
  'IsRedAlliance': True,
  'MatchNumber': 0.0,
  'MatchType': 0.0,
  'ReplayNumber': 0.0,
  'StationNumber': 1.0},
 'LiveWindow': {'.status': {'LW Enabled': False},
  'Ungrouped': {'.type': 'LW Subsystem',
   'AnalogInput[0]': {'.name': 'AnalogInput[0]',
    '.type': 'Analog Input',
    'Value': 0.0},
   'Climber': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
    '.hasDefault': False,
    '.name': 'Climber',
    '.type': 'Subsystem'},
   'Compressor[0]': {'.name': 'Compressor[0]',
    '.type': 'Compressor',
    'Enabled': False,
    'Pressure switch': False},
   'DifferentialDrive[1]': {'.actuator': True,
    '.name': 'DifferentialDrive[1]',
    '.type': 'DifferentialDrive',
    'Left Motor Speed': 0.0,
    'Right Motor Speed': 0.0},
   'Indexer': {'.command': 'none',
    '.default': 'none',
    '.hasCommand': False,
    '.hasDefault': False,
    '.name':

could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address
could not resolve roboRIO-2429-FRC.local address


In [285]:
def depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [286]:
depth(d)

6

In [291]:
depth(d['SmartDashboard']['intake piston'])

1

In [345]:
widget_dict = {'qlabel_camera_view': {'widget':'self.qlabel_camera_view', 'table':''},
        'qlabel_climber_indicator':{'widget':'self.qlabel_climber_indicator', 'table':''},
        }

In [351]:
for key, value in widget_dict.items():
    print(key, value)

qlabel_camera_view {'widget': 'self.qlabel_camera_view', 'table': ''}
qlabel_climber_indicator {'widget': 'self.qlabel_climber_indicator', 'table': ''}
